In [71]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td
import matplotlib.pyplot as plt
from torchvision import transforms
from torchsummary import summary
import numpy as np
import cv2
import os
import math

In [82]:
class CompCarDataset(td.dataset.Dataset):
    def __init__(self, train):
        postfix1 = "train" if train else "test"

        mean = [0.4695975, 0.47956536, 0.49133955] # Caclulated using the training set
        std = [0.07418917, 0.0721986, 0.07223192] # Calculated using the training set

        self.positive_rates = torch.Tensor([0.0431, 0.2164, 0.4099, 0.1916, 0.0091, 0.0314, 0.0164, 0.0053, 0.0137,
        0.0495, 0.0053, 0.0083, 0.0000, 0.0113, 0.0411, 0.2049, 0.2914, 0.2202,
        0.1125, 0.0954, 0.0110, 0.0043, 0.0041, 0.0039])

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
            transforms.Pad((0, 121)),
            transforms.Resize((224, 224))
        ])

        self.path_image = "comp-cars/" + postfix1 + "/images"
        self.path_speed = "comp-cars/" + postfix1 + "/speed"
        self.path_type = "comp-cars/" + postfix1 + "/type"

        self.images = os.listdir(os.path.join(self.path_image))
        self.labels_speed = os.listdir(os.path.join(self.path_speed))
        self.labels_type = os.listdir(os.path.join(self.path_type))
        self.images.sort()
        self.labels_speed.sort()
        self.labels_type.sort()
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        imagefile = os.path.join(self.path_image, self.images[idx])
        speedfile = open(os.path.join(self.path_speed, self.labels_speed[idx]), "r")
        typefile = open(os.path.join(self.path_type, self.labels_type[idx]), "r")
        img = self.transform(cv2.imread(imagefile))
        spd = float(speedfile.read())
        typ = int(typefile.read())
        lbl = torch.zeros(24)
        lbl[typ - 1] = 1
        lbl[12 + math.floor((spd - 100)/(20))] = 1
        return img, lbl, spd


In [53]:
class BasicBlockResNet(nn.Module):
    def __init__(self, channels_in, channels_out, size_change):
        super().__init__()

        self.strides = (2, 2) if size_change else (1, 1)

        self.size_change = size_change

        self.activation = nn.ReLU()

        self.layers = nn.Sequential(
            nn.Conv2d(channels_in, channels_out, kernel_size=(3, 3), stride=self.strides, padding=(1, 1)),
            nn.BatchNorm2d(channels_out),
            nn.ReLU(),
            nn.Conv2d(channels_out, channels_out, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(channels_out)
        )

        self.resizing = nn.Sequential(
            nn.Conv2d(channels_in, channels_out, kernel_size=(1, 1), stride=self.strides),
            nn.BatchNorm2d(channels_out)
        )

    def forward(self, x):
        residual = x
        x = self.layers(x)
        if self.size_change:
            residual = self.resizing(residual)
        x += residual
        x = self.activation(x)
        return x

In [40]:
#Final Tensor needs to be of Length 24 (12 car types, and 12 speed zones)
class ResNet18(nn.Module):
    def __init__(self, channels_in, sizes=[64, 128, 256, 512], num_classes=24):
        super().__init__()

        self.first_layer = nn.Sequential(
            nn.Conv2d(channels_in, sizes[0], kernel_size=(7, 7), stride=(2, 2), padding=3),
            nn.BatchNorm2d(sizes[0]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=1)
        )

        self.middle_layers = nn.Sequential(
            BasicBlockResNet(sizes[0], sizes[0], False),
            BasicBlockResNet(sizes[0], sizes[0], False),
            BasicBlockResNet(sizes[0], sizes[1], True),
            BasicBlockResNet(sizes[1], sizes[1], False),
            BasicBlockResNet(sizes[1], sizes[2], True),
            BasicBlockResNet(sizes[2], sizes[2], False),
            BasicBlockResNet(sizes[2], sizes[3], True),
            BasicBlockResNet(sizes[3], sizes[3], False)
        )

        self.final_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(sizes[3], num_classes)
        )

    def forward(self, x):        
        x = self.first_layer(x)
        x = self.middle_layers(x)
        x = self.final_layer(x)
        return x

In [88]:
class DeepMARLoss(nn.Module):
    def __init__(self, positive):
        super().__init__()
        self.rates = positive
    
    def forward(self, prediction, label):
        output_shape = prediction.shape
        loss = 0
        for i in range(output_shape[0]):
            for j in range(output_shape[1]):
                yil = label[i, j]
                wl = self.loss_weight(j, yil)
                pil = self.output_probability(prediction[i, j])
                iter_loss = wl * ((yil * math.log(pil)) + ((1 - yil)* math.log(1 - pil)))
                loss += iter_loss
        
        return torch.tensor(((-loss)/(output_shape[0])))
    
    def loss_weight(self, attr_index, label_val, loss_tuning=1):
        if label_val == 1:
            return math.exp((1 - self.rates[attr_index])/(math.pow(loss_tuning, 2)))
        else:
            return math.exp((self.rates[attr_index])/(math.pow(loss_tuning, 2)))

    def output_probability(self, x):
        return 1 / (1 + math.exp(-x))
                

In [84]:
def get_dataloader(dataset, batch):
    dataloader = td.DataLoader(dataset, batch_size=batch,shuffle=True)
    return dataloader

In [90]:
dataset = CompCarDataset(True)
network = ResNet18(3)
loader = get_dataloader(dataset, 50)
#img, lbl, spd = dataset.__getitem__(0)	
#img = torch.unsqueeze(img, 0)	
#final_lbl = network.forward(img)
criterion = DeepMARLoss(dataset.positive_rates)
for i in range(1):
    for iter_num, data_batch in enumerate(loader):
        img, lbl, spd = data_batch
        final_lbl = network.forward(img)
        print(final_lbl.shape)
        loss = criterion(final_lbl, lbl)
        print(loss.shape)

torch.Size([50, 24])
torch.Size([])
torch.Size([50, 24])
torch.Size([])


KeyboardInterrupt: 